In [1]:
import nltk
from collections import Counter
import re
from numpy import array
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to C:\Users\Sherif
[nltk_data]     Essam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sherif Essam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to C:\Users\Sherif
[nltk_data]     Essam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [2]:
def words(text):
  return re.findall(r'\w+',text.lower())

code for generating the bigrams

In [3]:
def generate_N_grams(text):
  l= [ (text[i],text[i+1]) for i in range(len(text)-1)]
  return l

here i initialize dictinaries for bigrams and frequncies and unigrams and their frequencies


In [4]:
bigrams=Counter(generate_N_grams(words(open('big.txt').read())))
My_dict=Counter(words(open('big.txt').read()))

FileNotFoundError: [Errno 2] No such file or directory: 'big.txt'

The traditional norvig spell check algorithm for any comparison with my improved changes

In [132]:
def edits(word):
  letters='qwertyuiopasdfghjklzxcvbnm'
  splits=[(word[:i],word[i:]) for i in range(len(word)+1)]
  d=[l+r[1:] for l,r in splits if len(r)!=0]
  t=[l+r[1]+r[0]+r[2:] for l,r in splits if len(r)>1]
  r=[l+i+r[1:] for l,r in splits if len(r)!=0 for i in letters]
  i=[l+i+r for l,r in splits  for i in letters]
  return set(d+t+r+i)
def edits2(word):
  return tuple(e2 for e1 in edits(word) for e2 in edits(e1))
def know(candidate):
  return list(c for c in candidate if c in My_dict)
# as for the candidates function i now give same priority and probability for both edits one and two
def bigram_candidates(word):
  return ( know([word]) or know(edits(word)) + know(edits2(word)) or [word])

In [133]:
def probability(word,n=sum(My_dict.values())):
 return My_dict[word]/n
def old_candidates(word):
  return (know([word]) or know(edits(word)) or know(edits2(word)) or [word])
def old_correction(word):
  return max(set(old_candidates(word)),key=probability)

In [177]:
My_dict['escabeing']

0

In [236]:
bigrams['the','it']

0

My Error model now dependes on the the context taking the conditional probability between the assumed misspelled word and the the word before it + probability between the assumed misspelled word and the the word after it taking the max value

In [249]:
def leftproba(word_before,Target):
  if My_dict[word_before]!=0:
    return 200*bigrams[word_before,Target]/My_dict[word_before]
  else:
   return 0
def rightproba(Target,word_after):
  if My_dict[word_after]!=0:
    return 50*bigrams[Target,word_after]/My_dict[word_after]
  else:
   return 0
def totalproba(left,word,right):
  return leftproba(left,word)+rightproba(word,right)

this is the block of code used for highlight text in google colab


In [135]:
from IPython.display import HTML as html_print
def cstr(s, color='white'):
    return "<text style=color:{}>{}</text>".format(color, s)
left, word, right = 'foo' , 'abc' , 'bar'
word=cstr(word, color='red')
html_print(cstr(' '.join([left, word, right]), color='white') )

This function is for spell checking each word in the given sentence using bigrams special case for fist and last word of sentence

In [136]:
def correction_bigrams(sentence):
  words=sentence.split()
  for i in range(len(words)):
    l=list(set(bigram_candidates(words[i])))
    max=0
    f=0
    if(len(l)==1):
      words[i]=l[0]
      continue
    for j in range(len(l)):
      if i==0:
        temp=rightproba(l[j],words[i+1])#first word in sentence
      elif i==len(words)-1:
        temp=leftproba(words[i-1],l[j])#last word
      else :
        temp=totalproba(words[i-1],l[j],words[i+1])
      if temp>max:
        max=temp
        f=1
        words[i]=l[j]
    if f==0:
      words[i]=old_correction(words[i])
  return words    

In [148]:
def  varient_norvig_spellcheck(sentence,func=correction_bigrams):
  original_words=sentence.split()
  assumed_corrected=func(sentence)
  for i in range(len(original_words)):
    if(original_words[i]!=assumed_corrected[i]):
      original_words[i]=cstr(original_words[i], color='red')
      assumed_corrected[i]=cstr(assumed_corrected[i], color='green')

  original_words.append(cstr('>>>'))
  total_display=original_words+assumed_corrected
  return total_display

next blocks of are naive evluation between old correction and bigrams correction

In [138]:
old_correction('loveed')

'loved'

In [98]:
old_correction('blaand')

'bland'

In [197]:
old_correction('dte')

'due'

In [349]:
correction_bigrams('the cta')

['the', 'city']

here i introduce the color feature highlight mis spelled words red and corrected with green

In [150]:
sentence='very blaad foor heaar'
output=varient_norvig_spellcheck(sentence)
html_print(cstr(' '.join(output)))

In [141]:
sentence='tht he waass'
output=varient_norvig_spellcheck(sentence)
html_print(cstr(' '.join(output)))

In [139]:
sentence='he loveed at'
output=varient_norvig_spellcheck(sentence)
html_print(cstr(' '.join(output)))

as concluded bigrams now play a role in deciding the best cadidate depending on context but there are more improvements to be made


# Now trying to improve performance using ***symspell*** algorithm for faster candidates retrieval

In [1]:
max_edit_distance =2 #to mimic the candidate generation of the old norvig approach
def get_deletes_list(w):
  "given a word, derive strings with up to max_edit_distance characters deleted"
  deletes = []
  queue = [w]
  for d in range(max_edit_distance):
      temp_queue = []
      for word in queue:
          if len(word) > 1:
              for c in range(len(word)):  # character index
                  word_minus_c = word[:c] + word[c + 1:]
                  if word_minus_c not in deletes:
                      deletes.append(word_minus_c)
                  if word_minus_c not in temp_queue:
                      temp_queue.append(word_minus_c)
      queue = temp_queue

  return deletes

Now dictionary for the performance boost

In [2]:
symspell_dict = {}

In [3]:
def create_symspell_dict_entry(w):
  longest_word_length = 0
  '''add word and its derived deletions to symspell_dict'''
  # check if word is already in symspell_dict
  # symspell_dict entries are in the form: (list of suggested corrections,frequency of word in corpus)
  new_word = False
  if w in symspell_dict:
      # increment count of word in corpus
      symspell_dict[w] = (symspell_dict[w][0], symspell_dict[w][1] + 1)
  else:
      symspell_dict[w] = ([], 1)
      longest_word_length = max(longest_word_length, len(w))

  if symspell_dict[w][1] == 1:
      # first appearance of word in corpus
      new_word = True
      deletes = get_deletes_list(w)
      for item in deletes:
          if item in symspell_dict:
              # add (correct) word to delete's suggested correction list
              symspell_dict[item][0].append(w)
          else:
              # note frequency of word in corpus is not incremented
              symspell_dict[item] = ([w], 0)

  return new_word,longest_word_length

In [4]:
def create_dictionary(fname):
        total_word_count = 0
        unique_word_count = 0
        dict_longest_word=0

        with open(fname) as file:
            for line in file:
                # separate by words by non-alphabetical characters
                words = re.findall('[a-z]+', line.lower())
                for word in words:
                    total_word_count += 1
                    unique,longest_word_generated=create_symspell_dict_entry(word)
                    if unique:
                        unique_word_count += 1
                    dict_longest_word=max(dict_longest_word,longest_word_generated)

        print("total words processed: %i" % total_word_count)
        print("total unique words in corpus: %i" % unique_word_count)
        print("total items in dictionary (corpus words and deletions): %i" % len(symspell_dict))
        print("  edit distance for deletions: %i" % max_edit_distance)
        print("  length of longest word in corpus: %i" % dict_longest_word)

In [7]:
create_dictionary("/content/big.txt")

total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 848496
  edit distance for deletions: 2
  length of longest word in corpus: 18


Sneek peek of what The SymSpell Dictionary stores

In [350]:
c=0
for key, value in symspell_dict.items():
  if c<25:
    print(key,value)
    c+=1

the (['other', 'these', 'they', 'there', 'those', 'their', 'threw', 'then', 'them', 'three', 'thief', 'lithe', 'theft', 'wythe', 'thee', 'theme', 'ether', 'bathe', 'lathe', 'tache', 'thwee', 'thine', 'theah', 'ethel'], 80030)
he (['the', 'when', 'she', 'have', 'her', 'they', 'home', 'head', 'then', 'shoe', 'help', 'hear', 'here', 'them', 'hope', 'hare', 'held', 'hole', 'huge', 'heel', 'hide', 'herd', 'heed', 'hate', 'haze', 'heap', 'shed', 'hyde', 'heh', 'hers', 'hemp', 'heat', 'howe', 'hero', 'hume', 'hoes', 'hire', 'heal', 'thee', 'hise', 'hale', 'heir', 'hue', 'hen', 'hive', 'cher', 'hey', 'dahe', 'hew', 'hem', 'herb', 'ache', 'chef', 'herr', 'hewn', 'heah', 'hell', 'whew', 'zheg', 'chew', 'shew', 'helm'], 12401)
te (['the', 'date', 'take', 'they', 'late', 'time', 'then', 'tell', 'true', 'note', 'step', 'tore', 'them', 'tie', 'ten', 'tide', 'gate', 'test', 'type', 'toe', 'fate', 'rate', 'tea', 'tale', 'site', 'tree', 'text', 'tire', 'tear', 'item', 'bite', 'kate', 'hate', 'tied', 'a

A basic edit distance algorithm between two strings focuses on candidates by inseration , replacement and adjecent characters transposition
Note symspell approach uses only deletion diffrences

In [8]:
def my_edit_distance(s1,s2):
  s1,s2=' '+s1,' '+s2
  m=array([0]*len(s1)*len(s2)).reshape(len(s1),len(s2))
  for i in range(len(s1)):
    for j in range(len(s2)):
      if j==0:
          m[i][j]=i
      elif i==0:
          m[i][j]=j
      else:
        m[i][j]=min(m[i-1][j]+1,m[i][j-1]+1,m[i-1][j-1]+(s1[i]!=s2[j]))
        
      # Taking the touch of Damerau Levenshtein varient were (transposition) of two adjecent characters is of cost 1
      if (i > 0 and j > 0 and s1[i] == s2[j - 1] and s1[i - 1] == s2[j] and s1[i] != s2[j]):
                    m[i][j] = min( m[i][j], m[i-2][j-2]+1)

  return m[-1][-1]

In [187]:
def symspell_candidates(string, silent=False,verbose=1):
  #verbose=0 error model chooses the old way highest occurence of an edit 1 one
  #verbose=1 error has edits 1 and 2 as candidates
  #verbose=3 error has max edit candidates
  longest_word_length=18
  "return list of suggested corrections for potentially incorrectly spelled word"
  if (len(string) - longest_word_length) > max_edit_distance:
      if not silent:
          print("no items in dictionary within maximum edit distance")
      return string

  suggest_dict = {}
  min_suggest_len = 6
  if(verbose==1):
    min_suggest_len = 2

  queue = [string]
  q_dictionary = {}  # items other than string that we've checked

  while len(queue) > 0:
      q_item = queue[0]  # pop
      queue = queue[1:]

      if (q_item in symspell_dict) and (q_item not in suggest_dict):
          if symspell_dict[q_item][1] > 0:
              assert len(string) >= len(q_item)
              suggest_dict[q_item] = (symspell_dict[q_item][1],len(string) - len(q_item))
              if (verbose == 0) and (len(string) == len(q_item)):
                  break
              elif (len(string) - len(q_item)) < min_suggest_len and verbose==0:
                    min_suggest_len = len(string) - len(q_item)
          for sc_item in symspell_dict[q_item][0]:
              if sc_item not in suggest_dict:
                  assert len(sc_item) > len(q_item)
                  assert len(q_item) <= len(string)
                  if len(q_item) == len(string):
                      assert q_item == string
                      item_dist = len(sc_item) - len(q_item)
                  assert sc_item != string
                  #computing the candidates of changes other than deletion
                  item_dist = my_edit_distance(sc_item, string)
                  if (verbose < 2) and (item_dist > min_suggest_len):
                      pass
                  elif item_dist <= max_edit_distance :
                      assert sc_item in symspell_dict  # should already be in symspell_dict if in suggestion list
                      suggest_dict[sc_item] = (symspell_dict[sc_item][1], item_dist)
                      if item_dist < min_suggest_len and verbose==0:
                            min_suggest_len = item_dist
                  if verbose < 2:
                      suggest_dict = {k: v for k, v in suggest_dict.items() if v[1] <= min_suggest_len}

      assert len(string) >= len(q_item)
      if (verbose < 2) and ((len(string) - len(q_item)) > min_suggest_len):
          pass
      elif (len(string) - len(q_item)) < max_edit_distance and len(q_item) > 1:
          for c in range(len(q_item)):  # character index
              word_minus_c = q_item[:c] + q_item[c + 1:]
              if word_minus_c not in q_dictionary:
                  queue.append(word_minus_c)
                  q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this
  
  # queue is now empty: convert suggestions in dictionary to list for output
  if not silent and verbose != 0:
      print("number of possible corrections: %i" % len(suggest_dict))
      print("  edit distance for deletions: %i" % max_edit_distance)

  # return list of suggestions with (correction,(frequency in corpus, edit distance)):
  outlist = list(suggest_dict.items())
  if len(outlist)==0:
    return string

  if verbose == 0:
      outlist = sorted(outlist, key=lambda x: (x[1][1], -x[1][0]))
      return outlist[0][0]
  else:
      return outlist

def symspell_correction(s, silent=True):
    try:
        return symspell_candidates(s, silent,verbose=0)
    except:
        return s

Now i would like to demonstrate perfomance wise between symspell and norvig algorithms Next is a code block taken from [Norvig spell check website](https://norvig.com/spell-correct.html) in the Evaluation section with a [small data](https://norvig.com/spell-testset1.txt) set for accuracy and perfomance measure

In [126]:
def spelltest(tests,func):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.process_time()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = func(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in My_dict)
    dt = time.process_time() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} WORDS per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    """Parse 'right: wrong1 wrong2' lines into 
    [('right', 'wrong1'), ('right', 'wrong2')] pairs."""
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print('The old norvig')
spelltest(Testset(open('/content/spell-testset1.txt')),old_correction)

print('The optimized symspell technique')
spelltest(Testset(open('/content/spell-testset1.txt')),symspell_correction)

The old norvig
75% of 270 correct (6% unknown) at 26 WORDS per second 
The optimized symspell technique
75% of 270 correct (6% unknown) at 152 WORDS per second 


Conclusion same result for both algorithms but pre calculation of about 5 seconds once at the beginning for symspell is worth for the perfomance boost ***26 up to 152 words per second*** that is gives for the spell checker processing helping in producing a complex algorithm for the accuracy increase as well

## Next combining symspell and bigrams techniques together

In [ ]:
symspell_candidates('is',silent=True)

In [231]:
def symspell_bigrams(sentence):
  words=sentence.split()
  for i in range(len(words)):
    ##############
    l=symspell_candidates(words[i],silent=True)
    ##############
    max=0
    f=0
    if type(l)==str or l[0][0]== words[i]:
      continue
    for j in range(len(l)):
      if i==0:
        temp=rightproba(l[j][0],words[i+1])#first word in sentence
      elif i==len(words)-1:
        temp=leftproba(words[i-1],l[j][0])#last word
      else :
        temp=totalproba(words[i-1],l[j][0],words[i+1])
      if temp>max:
        max=temp
        f=1
        words[i]=l[j][0]
    if f==0:
      ##########
      words[i]=symspell_correction(words[i])
      ##########
  return words 

In [258]:
#Same correction approach using bigrams but with performance optimization of symspell
sentence='the cta is baetiful'
output=varient_norvig_spellcheck(sentence,symspell_bigrams)
html_print(cstr(' '.join(output)))

There are still false correction due to biased data set because the word **it** occurs more with **the** and **is** than **city** or **cat** model takes left and right bigrams to consideration

In [261]:
print(bigrams['the','it'],' ',bigrams['it','is'])#most frequent
print(bigrams['the','cat'],' ',bigrams['cat','is'])
print(bigrams['the','city'],' ',bigrams['city','is'])

0   1673
1   0
78   1


more over if we take only left bigrams in error calculation **cta** would be corrected to **city** the most frequent 

Next steps working on using pos tags to my advantage next i have done some pos tagging analysis also would like to use bigger data sets the following is simple basic demo for demonstrating the effect of pos tags to be continued in future--

In [336]:
pos_tagged_dict=Counter(generate_N_grams(nltk.pos_tag(words(open('big.txt').read()),tagset='universal')))

here using the idea of bigrams i output the most commom sequences a pos tag before it and after as well

In [337]:
pos_tagged_dict #-----

Counter({(('the', 'DET'), ('project', 'NOUN')): 95,
         (('project', 'NOUN'), ('gutenberg', 'ADJ')): 7,
         (('gutenberg', 'ADJ'), ('ebook', 'NOUN')): 7,
         (('ebook', 'NOUN'), ('of', 'ADP')): 5,
         (('of', 'ADP'), ('the', 'DET')): 12528,
         (('the', 'DET'), ('adventures', 'NOUN')): 5,
         (('adventures', 'NOUN'), ('of', 'ADP')): 7,
         (('of', 'ADP'), ('sherlock', 'NOUN')): 7,
         (('sherlock', 'NOUN'), ('holmes', 'NOUN')): 74,
         (('holmes', 'NOUN'), ('by', 'ADP')): 3,
         (('by', 'ADP'), ('sir', 'NOUN')): 10,
         (('sir', 'NOUN'), ('arthur', 'NOUN')): 3,
         (('arthur', 'NOUN'), ('conan', 'NOUN')): 4,
         (('conan', 'NOUN'), ('doyle', 'ADJ')): 3,
         (('doyle', 'ADJ'), ('15', 'NUM')): 1,
         (('15', 'NUM'), ('in', 'ADP')): 1,
         (('in', 'ADP'), ('our', 'PRON')): 82,
         (('our', 'PRON'), ('series', 'NOUN')): 2,
         (('series', 'NOUN'), ('by', 'ADP')): 2,
         (('by', 'ADP'), ('sir', 'A

In [315]:
for i in ['VERB', 'DET', 'NOUN', 'ADJ', 'ADV', 'ADP', 'CONJ', 'PRON', 'NUM', 'PRT','X'] :
  left_bigram = nltk.FreqDist( a[1] for (a, b) in pos_tagged_dict if b[1]==i)
  right_bigram = nltk.FreqDist( b[1] for (a, b) in pos_tagged_dict if a[1]==i)
  print(left_bigram.max()+" -> '"+i+"' -> "+right_bigram.max())

NOUN -> 'VERB' -> VERB
NOUN -> 'DET' -> NOUN
ADJ -> 'NOUN' -> VERB
VERB -> 'ADJ' -> NOUN
NOUN -> 'ADV' -> VERB
NOUN -> 'ADP' -> NOUN
NOUN -> 'CONJ' -> NOUN
VERB -> 'PRON' -> NOUN
NOUN -> 'NUM' -> NOUN
NOUN -> 'PRT' -> VERB
NOUN -> 'X' -> NOUN


some basic examples to show were pos tagging comes in handy

In [263]:
s=nltk.pos_tag(['They', 'where', 'going'],tagset='universal')
s
#where----->were(best candidate)

[('They', 'PRON'), ('where', 'ADV'), ('going', 'VERB')]

In [309]:
s=nltk.pos_tag(['he', 'were', 'tall'],tagset='universal')
s
#were----->was(best candidate)

[('he', 'PRON'), ('were', 'VERB'), ('tall', 'ADJ')]

In [342]:
s=nltk.pos_tag(['the','man', 'he', 'going'],tagset='universal')
s
#he----->is(best candidate)

[('the', 'DET'), ('man', 'NOUN'), ('he', 'PRON'), ('going', 'VERB')]

In [ ]:
s=nltk.pos_tag(['the','cta'],tagset='universal')
s

A start Basc idea to use naive approach to take frequency of pos of tags in the probalistic model in a start case two (bigrams)

In [345]:
def symspell_bigrams_pos(sentence):
  words=sentence.split()
  s=nltk.pos_tag(words,tagset='universal')
  for i in range(len(s)):
    ##############
    l=symspell_candidates(s[i][0],silent=True)
    ##############
    max=0
    f=0
    if type(l)==str:
      continue
    for j in range(len(l)):
      freq=0
      if i==0 or i==len(s)-1:
        continue
      else :#calculation of occurence of a candidated given the pos tags of words before and after
        for a,b in pos_tagged_dict:
          if (b[0]==l[j][0] and a[1]==s[i-1][1]) or(a[0]==l[j][0] and b[1]==s[i+1][1]) :
            freq+=1
        temp=freq
      if temp>max:
        max=temp
        f=1
        words[i]=l[j][0]
        s[i]=nltk.pos_tag([l[j][0]],tagset='universal')[0]
    if f==0:
      ##########
      words[i]=symspell_correction(words[i])
      ##########
  return s 

Started by simple idea considering the one incorrect word and its position in the middle

In [341]:
symspell_bigrams_pos('a contry walk')

[('a', 'DET'), ('country', 'NOUN'), ('walk', 'NOUN')]

In [343]:
symspell_bigrams('a contry walk')

['a', 'country', 'walk']

In [319]:
symspell_bigrams('they where going')#with out pos tagging

['they', 'where', 'going']

As excpeted the correction depends on miss spelling not grammatic error so no change

In [346]:
symspell_bigrams_pos('they where going')#with pos tagging

[('they', 'PRON'), ('were', 'VERB'), ('going', 'VERB')]

A good start to show the role pos tags can have choosing the best candidate not just in spelling but in context with help of bigrams

Still many obtcales to overcome: 

1. what if wrong verb assigned to wrong pronoun he were -> he was
2. what if we has missing words other than misspelled for sense meaning?
3. what if fisrt of last word are in worng context?
4. what if sentence has no spelling mistakes but words are in     disorder in sentence?

This is just a beginning and pos tags are for sure important is making more complex models (ML,DL)

For Futher development......


 